# Bài 1: Canny Edge Detection

### The Canny edge detection algorithm is composed of 5 steps:

- Noise reduction
- Gradient calculation
- Non-maximum suppression
- Double threshold
- Edge Tracking by Hysteresis

In [ ]:
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

# img = cv2.imread('messi5.jpg',0)
# edges = cv2.Canny(img,100,200)

# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edges,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

# plt.show()

# Bài 2: Hough Line Transform & Hough Circle Transform

## 2.1 Hough Line Transform

- "Basic" Hough Transform
- Probabilistic Hough Transform

As we learned from quite early school classes, the straight line can be represented by two parameters. The simplest and most widely used pair of parameters is (a, b) which correspond to slope and intercept. The line is then described as: y = a x + b
Let’s forget for a while about these parameters. We can also unambiguously describe the line using the pair (ρ, θ) in polar system. The first parameter, ρ, is the shortest distance from the origin to the line (approaching the line perpendicularly). The second, θ, is the angle between x-axis and the distance line. One of the benefits of such representation is that we can describe vertical lines by ρ and θ which is impossible by using only (a, b) parameters in Cartesian system.

<img src="houghtransform_mapping.jpg">

For a given line, we can determine specific ρ and θ. Then, the following equation is satisfied for each x, y point belonging to this line:
__ρ = x cos(θ) + y sin(θ)__

<img src="houghtransformlines.jpg">

In [14]:
import cv2
import numpy as np

img = cv2.imread('sudoku.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)

lines = cv2.HoughLines(edges,1,np.pi/180,200)
print(lines[1])
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imwrite('houghlines3.jpg',img)

[[268.          1.5707964]]


True

## 2.2 Hough Circle Transform

In [1]:
# import numpy as np
# import cv2 as cv
# img = cv.imread('opencv-logo-white.png',0)
# img = cv.medianBlur(img,5)
# cimg = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
# circles = cv.HoughCircles(img,cv.HOUGH_GRADIENT,1,20,
#                             param1=50,param2=30,minRadius=0,maxRadius=0)
# circles = np.uint16(np.around(circles))
# for i in circles[0,:]:
#     # draw the outer circle
#     cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
#     # draw the center of the circle
#     cv.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

# Bài 3: Image Pyramids

## What are image pyramids?

<img src="pyramid_example.png">

An “image pyramid” is a __multi-scale representation__ of an image.

Utilizing an image pyramid allows us to __find objects in images at different scales__ of an image. And when combined with a sliding window we can __find objects in images in various locations__.

At the bottom of the pyramid we have the original image at its original size (in terms of width and height). And at each subsequent layer, the image is resized (subsampled) and optionally smoothed (usually via Gaussian blurring).

The image is progressively subsampled until some stopping criterion is met, which is normally a minimum size has been reached and no further subsampling needs to take place.

# Bài 4: Image Segmentation with Watershed Algorithm

Any grayscale image can be viewed as a topographic surface where high intensity denotes peaks and hills while low intensity denotes valleys. You start filling every isolated valleys (local minima) with different colored water (labels). As the water rises, depending on the peaks (gradients) nearby, water from different valleys, obviously with different colors will start to merge. To avoid that, you build barriers in the locations where water merges. You continue the work of filling water and building barriers until all the peaks are under water. Then the barriers you created gives you the segmentation result. This is the “philosophy” behind the watershed. You can visit the CMM webpage on watershed to understand it with the help of some animations.

http://www.cmm.mines-paristech.fr/~beucher/wtshed.html

But this approach gives you oversegmented result due to noise or any other irregularities in the image. So OpenCV implemented a marker-based watershed algorithm where you specify which are all valley points are to be merged and which are not. It is an interactive image segmentation. What we do is to give different labels for our object we know. Label the region which we are sure of being the foreground or object with one color (or intensity), label the region which we are sure of being background or non-object with another color and finally the region which we are not sure of anything, label it with 0. That is our marker. Then apply watershed algorithm. Then our marker will be updated with the labels we gave, and the boundaries of objects will have a value of -1.

